In [11]:
import os
import cv2
import time
import shutil
import keyboard
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageGrab, Image
from pynput.keyboard import Key, Controller

kb = Controller()
lst = {0:'w', 1:'a', 2:'s', 3:'d', 4:'nop'}

In [16]:
def clean():
    shutil.rmtree(r'data')
    os.system('mkdir data')
    os.chdir(os.getcwd() + '\\data')
    for i in range(5):
        os.system('mkdir {}'.format(lst[i]))
    os.chdir('..')
    return 1

def move(opt):
    if opt < 4:
        opt = lst[opt]
        hit_key(opt)

def hit_key(key):
    kb.press(key)
    time.sleep(0.09)
    kb.release(key)

# def roi(img, vertices):
#     mask = np.zeros_like(img)
#     cv2.fillPoly(mask, vertices, 255)
#     masked = cv2.bitwise_and(img, mask)
#     return masked

def process_img(original_image):
    processed_img = original_image
    # Grey scale
    processed_img = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    # Zeropadding the image
    processed_img = np.pad(processed_img, ((0,144),(0,0)), 'constant')
    # ROI
#     vertices = np.array([[440,37],[158,597], [425,664], [1001,665], [1229,247]], np.int32)
#     processed_img = roi(processed_img, [vertices])
    # Straighten the image
    pts_src = np.array([[440,37], [1229,247], [158,597],[947,807]])
    pts_dst = np.array([[0,0],[817, 0],[0,627],[817, 627]])
    im_dst = np.zeros((627, 817, 3), np.uint8)
    h, status = cv2.findHomography(pts_src, pts_dst)
    processed_img = cv2.warpPerspective(processed_img, h, (im_dst.shape[1],im_dst.shape[0]))
    
    return processed_img

In [14]:
clean()

1

In [17]:
start = 0
img_num = 0

tmp = np.eye(5, dtype=int)

# %b  %d %H %M
# Jul 18 12:10
t = time.strftime("%b%d_%H%M_", time.localtime())
print('Current time is: ', t[:-1])

for i in range(3):
    print('{}...'.format(3-i))
    time.sleep(1)

while True:
    screen =  np.array(ImageGrab.grab(bbox=(0,33,1280,699)))
    new_screen = process_img(screen)
    cv2.imshow('monitor', new_screen)
    
    p = 4
    for i in range(4):
        if keyboard.is_pressed(lst[i]): 
            p = i
            if not start:
                print('started')
                start = 1
            time.sleep(0.2)
    
    img_id = t + str(img_num)
    if start == 1:
        Image.fromarray(new_screen).save('data/{}/{}.jpg'.format(lst[p], img_id))
        img_num += 1
    if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

Current time is:  Jul25_0113
3...
2...
1...
started
